<a href="https://colab.research.google.com/github/namratabiswas/Attention_in_Transformers/blob/main/masked_token_generation_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install dependencies
!pip install transformers torch bertviz

In [ ]:
# Import libraries

import torch
from transformers import BertTokenizer, BertForMaskedLM
from bertviz import head_view
from IPython.display import display, HTML

In [ ]:
# Load BERT tokenizer and model (Encoder-only)
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name, output_attentions=True)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
# Input sentence with a [MASK] token for generation
sentence = "The quick brown fox jumps over the [MASK] dog."
inputs = tokenizer(sentence, return_tensors='pt')
mask_index = torch.where(inputs['input_ids'][0] == tokenizer.mask_token_id)[0].item()


In [ ]:
# Forward pass to get predictions and attentions
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    attentions = outputs.attentions  # shape: [layers x batch x heads x tokens x tokens]

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [ ]:
# Get top 5 predicted tokens for the [MASK]
mask_token_logits = logits[0, mask_index]
top_tokens = torch.topk(mask_token_logits, 5, dim=0).indices

print("Top 5 predictions for [MASK]:")
for token in top_tokens:
    print(f" - {tokenizer.decode([token])}")

Top 5 predictions for [MASK]:
 - little
 - big
 - small
 - startled
 - sleeping


In [ ]:
# Replace [MASK] with top predicted token
predicted_token = tokenizer.decode([top_tokens[0]])
generated_sentence = sentence.replace("[MASK]", predicted_token)
print("\nGenerated Sentence:\n", generated_sentence)


Generated Sentence:
 The quick brown fox jumps over the little dog.


In [ ]:
# Visualize attention
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
head_view(attentions, tokens)

<IPython.core.display.Javascript object>